# **Affective Signals - I**

*Through this notebook we hope you get an hands on approach to the seminar. Below you will find code snippets and theory that will help you apply the libraries that will be introduced to you in the course.*

 # **First of all, please copy this Notebook in your Drive.** 

---


 Otherwise you are changing our code and we can see your data.

####**Install OpenFace**
The first code snippet installs all the necessary dependencies and may take a while (30~40 minutes). Hence, everytime you close this notebook, you need to install all dependencies again to work on the notebook. Furthermore you need a Google Account, obviously.

---

During the installation, you will be asked about a configuration file, just type "N" for default action, and the installation will proceed. We added a sound hint shortly when you are asked for input, as well as you will hear a sound when the installation is done. 

In [ ]:

import os
from os.path import exists, join, basename, splitext
from IPython.display import Audio
sound_file = 'https://www.soundjay.com/buttons/sounds/button-2.mp3'

################# Need to revert back to CUDA 10.0 ##################
# Thanks to http://aconcaguasci.blogspot.com/2019/12/setting-up-cuda-100-for-mxnet-on-google.html
#Uninstall the current CUDA version
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

#Download CUDA 10.0
!wget  --no-clobber https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/cuda-repo-ubuntu1804_10.0.130-1_amd64.deb
Audio(sound_file, autoplay=True)
#install CUDA kit dpkg
!dpkg -i cuda-repo-ubuntu1804_10.0.130-1_amd64.deb
!sudo apt-key adv --fetch-keys https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64/7fa2af80.pub
!apt-get update
!apt-get install cuda-10-0
#Slove libcurand.so.10 error
!wget --no-clobber http://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64/nvidia-machine-learning-repo-ubuntu1804_1.0.0-1_amd64.deb
#-nc, --no-clobber: skip downloads that would download to existing files.
!apt install ./nvidia-machine-learning-repo-ubuntu1804_1.0.0-1_amd64.deb
!apt-get update
####################################################################

git_repo_url = 'https://github.com/TadasBaltrusaitis/OpenFace.git'
project_name = splitext(basename(git_repo_url))[0]n


# clone openface
!git clone -q --depth 1 $git_repo_url

# install new CMake becaue of CUDA10
!wget -q https://cmake.org/files/v3.13/cmake-3.13.0-Linux-x86_64.tar.gz
!tar xfz cmake-3.13.0-Linux-x86_64.tar.gz --strip-components=1 -C /usr/local

# Get newest GCC
!sudo apt-get update
!sudo apt-get install build-essential 
!sudo apt-get install g++-8

# install python dependencies
#!pip install -q youtube-dl

# Finally, actually install OpenFace
!cd OpenFace && bash ./download_models.sh && sudo bash ./install.sh
Audio(sound_file, autoplay=True)

In [ ]:
# mount your Google Drive, where your video is located

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Neuer Abschnitt

In [ ]:
import numpy as np
import pandas as pd

import subprocess
import time
import os

import matplotlib.pyplot as plt 

import re
import seaborn as sns

# Facial Expression

We tend to use facial expressions to communicate with each other and also express our emotional state. A facial expression is one or more motions or positions of the muscles beneath the skin of the face. These movements are believed convey the emotional state of an individual to observers. Thus, facial expressions are one form of nonverbal communication.[Source](https://en.wikipedia.org/wiki/Facial_expression)


## [OpenFace](https://github.com/TadasBaltrusaitis/OpenFace/wiki)

This tool will be extensively used in this notebook to extract relevant information from your videos. Below you will find code snippets to process your videos using the OpenFace - Library that you were introduced to in the seminar.

In [ ]:
#path to your videos should be added here

#videos = "path to your video"

# e.g.
videos = "/content/drive/MyDrive/AffectiveSignals/video.mp4"

# create a subfolder for your video directory called e.g. "ExtractedFeatures"
output_dir = "/content/drive/MyDrive/AffectiveSignals/ExtractedFeatures/"

In [ ]:
# Run OpenFace on your Video

!./OpenFace/build/bin/FeatureExtraction -f "$videos" -out_dir "$output_dir"


---

You can find the processed video in the directory that you ran the jupyter-notebook, you can import the processed video and play it as shown below.






---







In [ ]:
# Display video with extracted features

def show_local_mp4_video(file_name, width=640, height=480):
  import io
  import base64
  from IPython.display import HTML
  video_encoded = base64.b64encode(io.open(file_name, 'rb').read())
  return HTML(data='''<video width="{0}" height="{1}" alt="test" controls>
                        <source src="data:video/mp4;base64,{2}" type="video/mp4" />
                      </video>'''.format(width, height, video_encoded.decode('ascii')))

#covert the processed file to mp4 so it can be displayed in a browser (it takes a couple of seconds)

# e.g.
!ffmpeg -y -loglevel info -i "/content/drive/MyDrive/AffectiveSignals/ExtractedFeatures/video.avi" "/content/drive/MyDrive/AffectiveSignals/ExtractedFeatures/video.mp4"

#!ffmpeg -y -loglevel info -i "your_avi_file_directory" "your_output_dir"

# display video

# e.g
show_local_mp4_video("/content/drive/MyDrive/AffectiveSignals/ExtractedFeatures/1620144600921.mp4", width=960, height=720)

#show_local_mp4_video("path to your processed video", width=960, height=720)

----
OpenFace also creates a csv file that contains all features that have been extracted from the video.


---



In [ ]:
#load the csv file using pandas
# e.g
df_clean = pd.read_csv("/content/drive/MyDrive/AffectiveSignals/ExtractedFeatures/video.csv")

#df_clean = pd.read_csv("path to your generated csv file")

In [ ]:
#Short view of all the contents of the file 
df_clean.head()

#### **Action Units**
Facial Action Coding System (FACS) is a system to taxonomize human facial movements by their appearance on the face. Movements of individual facial muscles are encoded by FACS from slight different instant changes in facial appearance. Using FACS it is possible to code nearly any anatomically possible facial expression, deconstructing it into the specific Action Units (AU) that produced the expression. It is a common standard to objectively describe facial expressions [[1]](https://github.com/TadasBaltrusaitis/OpenFace/wiki/Action-Units)

List of all the Action units can be found [here](https://www.cs.cmu.edu/~face/facs.htm). You can find small animation of the action units [here](https://imotions.com/blog/facial-action-coding-system/).



---

Let's look at the intensity plots of all the action units

In [ ]:
# Threshold data by 80%
df_clean = df_clean[df_clean.confidence>=.80]
# Plot all Action Unit time series. 
au_regex_pat = re.compile(r'^AU[0-9]+_r$')
au_columns_r = df_clean.columns[df_clean.columns.str.contains(au_regex_pat)]
print("List of AU columns:", au_columns_r)
f,axes = plt.subplots(6, 3, figsize=(10,12), sharex=True, sharey=True)
axes = axes.flatten()
for au_ix, au_col in enumerate(au_columns_r):
    sns.lineplot(x='frame', y=au_col, hue='face_id', data=df_clean, ax=axes[au_ix])
    axes[au_ix].set(title=au_col, ylabel='Intensity')
    axes[au_ix].legend(loc=5)
plt.suptitle("AU intensity predictions", y=1.02)
plt.tight_layout()

In [ ]:
au_regex_pat = re.compile(r'^AU[0-9]+_c$')
au_columns = df_clean.columns[df_clean.columns.str.contains(au_regex_pat)]
print("List of AU columns:", au_columns)
f,axes = plt.subplots(6, 3, figsize=(10,12), sharex=True, sharey=True)
axes = axes.flatten()
for au_ix, au_col in enumerate(au_columns):
    sns.lineplot(x='frame', y=au_col, hue='face_id', data=df_clean, ax=axes[au_ix])
    axes[au_ix].set(title=au_col, ylabel="Presence")
    axes[au_ix].legend(loc=5)
plt.suptitle("AU occurence predictions", y=1.02)
plt.tight_layout()



---


We know that AU45 corresponds to eye blinks. Let's try to find out how many times do you blink in your own video based on the presence of the according AU. Use the .csv provided by the OpenFace-Library!


---



**Task 1.1: Count the number of times the action unit AU45 occurs in your video**

AU12 and AU6 correspond to happiness, so let's see how many times do they co-occur in your video:

---




**Task 1.2: Count the number of times the action units 12 and 6 are present simultaneously in your video**

In [ ]:
#your code goes here:

---

**Task 1.3: Visualize the correlation of all the Action Units that are present based on their intensity**

In [ ]:
#your code goes here (hint: try to use AU_columns_r variables to get the required columns and find the correlation matrix from the dataframe):



#once you get you corelation you can use the below function to plot the matrix
sns.heatmap(corr, 
            xticklabels=corr.columns.values,
            yticklabels=corr.columns.values)

---

**Task 1.4: Calculate the mean and variance for each of the Action Unit intensities**

In [ ]:
#your code goes here:

If you are interested, try out additional analyses and visualizations to explore your individual facial expressions in the video. For example, try to find out how much you're smiling in your video. We can discuss your findings and ideas in the upcoming seminar!

In [ ]:
#a place for additional interesting analyses:

**-----------------END OF FACIAL EXPRESSIONS-------------------**